In [2]:
from datasets import load_dataset

In [4]:
raw_dataset = load_dataset('kde4',lang1='en',lang2='zh_CN')

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 139666
    })
})

In [6]:
split_dataset = raw_dataset['train'].train_test_split(train_size=0.9,seed=20)

In [7]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 125699
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 13967
    })
})

In [11]:
split_dataset['train'][0]

{'id': '114557', 'translation': {'en': 'Download', 'zh_CN': '下载'}}

In [12]:
split_dataset['validation'] = split_dataset.pop('test')

In [13]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 125699
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 13967
    })
})

In [15]:
split_dataset['train'][1]['translation']

{'en': 'Installation prefix for Qt', 'zh_CN': 'Qt 的安装前缀'}

In [17]:
from transformers import pipeline

In [20]:
model_checkpoint = 'Helsinki-NLP/opus-mt-en-zh'

In [21]:
translator = pipeline('translation',model=model_checkpoint)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/806k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/805k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.62M [00:00<?, ?B/s]

/Users/liuchu/opt/anaconda3/envs/mem0/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use mps:0


In [28]:
translator('what is the fuck')

[{'translation_text': '什么是他妈的'}]

In [29]:
split_dataset['train'][172]['translation']

{'en': 'Quebec', 'zh_CN': '魁北克canada. kgm'}

In [30]:
translator('Quebec')

[{'translation_text': '魁北克省'}]

In [31]:
from transformers import AutoTokenizer

In [32]:
model_checkpoint = 'Helsinki-NLP/opus-mt-en-zh'

In [33]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")


/Users/liuchu/opt/anaconda3/envs/mem0/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [34]:
en_sentence = split_dataset['train'][1]['translation']['en']

In [36]:
zh_sentence = split_dataset['train'][1]['translation']['zh_CN']

In [38]:
inputs = tokenizer(en_sentence,text_target=zh_sentence)

In [39]:
inputs

{'input_ids': [54596, 2765, 594, 10110, 15, 8, 632, 60, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [8, 632, 60, 8, 12, 9613, 637, 56891, 0]}

In [41]:
tokenizer.convert_ids_to_tokens(inputs['input_ids'])

['▁Installation', '▁pre', 'f', 'ix', '▁for', '▁', 'Q', 't', '</s>']

In [42]:
max_length = 128

In [43]:
def preprocess_function(examples):
    inputs = [ex['en'] for ex in examples['translation']]
    targets = [ex['zh_CN'] for ex in examples['translation']]
    model_inputs = tokenizer(
       inputs,text_target=targets,max_length=max_length,truncation=True
    )
    return model_inputs

In [ ]:
preprocess_function(split_dataset['validation'])

In [45]:
tokenized_datasets = split_dataset.map(
  preprocess_function,
    batched=True,
    remove_columns=split_dataset['train'].column_names,
)

Map:   0%|          | 0/125699 [00:00<?, ? examples/s]

Map:   0%|          | 0/13967 [00:00<?, ? examples/s]

In [47]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 125699
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 13967
    })
})

In [50]:
split_dataset['train'].column_names,

(['id', 'translation'],)

model

In [52]:
from transformers import AutoModelForSeq2SeqLM

In [53]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [54]:
from transformers import DataCollatorForSeq2Seq

In [55]:
data_collator = DataCollatorForSeq2Seq(tokenizer,model=model)

In [56]:
batch = data_collator([tokenized_datasets['train'][i] for i in range(1,3)])

In [58]:
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [59]:
batch["labels"]


tensor([[    8,   632,    60,     8,    12,  9613,   637, 56891,     0],
        [    8, 46315,     0,  -100,  -100,  -100,  -100,  -100,  -100]])

In [60]:
batch["decoder_input_ids"]


tensor([[65000,     8,   632,    60,     8,    12,  9613,   637, 56891],
        [65000,     8, 46315,     0, 65000, 65000, 65000, 65000, 65000]])

In [61]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[8, 632, 60, 8, 12, 9613, 637, 56891, 0]
[8, 46315, 0]


In [64]:
import evaluate

metric = evaluate.load("sacrebleu")

In [63]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 10.4 kB/s eta 0:00:0031m10.4 kB/s eta 0:00:03
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 26.0 kB/s eta 0:00:000:00:0136m0:00:01:03
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)


In [65]:
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [66]:
predictions = ["This This This This"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 1.683602693167689,
 'counts': [1, 0, 0, 0],
 'totals': [4, 3, 2, 1],
 'precisions': [25.0, 16.666666666666668, 12.5, 12.5],
 'bp': 0.10539922456186433,
 'sys_len': 4,
 'ref_len': 13}

In [67]:
predictions = ["This plugin"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 0.0,
 'counts': [2, 1, 0, 0],
 'totals': [2, 1, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 0.004086771438464067,
 'sys_len': 2,
 'ref_len': 13}

In [68]:
import numpy as np

In [70]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decode_preds =  tokenizer.batch_decode(preds,skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_type_id)
    decode_labels = tokenizer.batch_decode(labels,skip_special_tokens=True)
    
    decode_preds = [pred.strip() for pred in decode_labels]
    decode_labels = [[label.strip()] for label in decode_labels]
    
    result = metric.compute(predictions=decode_preds,references=decode_labels)
    return {'blue':result['score']}

In [75]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    bf16=True,   # 开启 bf16
    push_to_hub=False,
)

In [76]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/var/folders/z2/kds62tbj3x93zghv2_jz1xsr0000gn/T/ipykernel_21758/543751272.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [77]:
trainer.evaluate(max_length=max_length)


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

...

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

{'eval_loss': 2.4871392250061035,
 'eval_model_preparation_time': 0.0026,
 'eval_blue': 100.00000000000004,
 'eval_runtime': 58216.7013,
 'eval_samples_per_second': 0.24,
 'eval_steps_per_second': 0.004}

In [78]:
trainer.train()

Step,Training Loss


KeyboardInterrupt: 

In [79]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [80]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 125699
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13967
    })
})

In [81]:
eval_dataloader

In [82]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

python(55976) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [83]:
model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(65001, 512, padding_idx=65000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(65001, 512, padding_idx=65000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [84]:
from transformers import AdamW

In [85]:
optimizer = AdamW(model.parameters(),lr=2e-5)

/Users/liuchu/opt/anaconda3/envs/mem0/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [86]:
optimizer

AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    lr: 2e-05
    weight_decay: 0.0
)

In [87]:
from accelerate import Accelerator

In [88]:
accelerator = Accelerator()

In [89]:
model,optimizer,train_dataloader,eval_dataloader = accelerator.prepare(
  model,optimizer,train_dataloader,eval_dataloader
)

In [92]:
from transformers import get_scheduler

In [93]:
num_train_epochs = 3

In [94]:
num_update_steps_per_epoch = len(train_dataloader)

In [95]:
num_update_steps_per_epoch

15713

In [96]:
num_training_steps = num_train_epochs * num_update_steps_per_epoch


In [97]:
num_training_steps

47139

In [98]:
lr_scheduler = get_scheduler(
   "linear",
   optimizer=optimizer,
   num_warmup_steps=0,
   num_training_steps=num_training_steps
)

In [99]:
lr_scheduler

In [100]:
def postprocess(predictions,labels):
    
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()
    
    decode_preds = tokenizer.batch_decode(predictions,skip_special_tokens=True)
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [101]:
from tqdm.auto import tqdm

In [102]:
import torch

In [103]:
progress_bar = tqdm(range(num_training_steps))

  0%|          | 0/47139 [00:00<?, ?it/s]

In [107]:
for epoch in range(num_train_epochs):
    
    model.train()
    
    for batch in train_dataloader:
        
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    
    
    model.eval()
    
    for batch in tqdm(eval_dataloader):
        
        with torch.no_grad():
            
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch['input_ids'],
                attention_mask=batch['attention_mask'],
                max_length=128,
            )
        labels = batch['labels']
        
        
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens,dim=1,pad_inde=tokenizer.pad_token_id
        )
        
        
        labels = accelerator.pad_across_processes(labels,dim=1,pad_index=-100)
        
        predictions_gathered = accelerator.gather(generated_tokens)
        
        labels_gathered = accelerator.gather(labels)
        
        
        
        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)
        
    results = metric.compute()
    
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")
    
        # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

